## Install All Packages Needed

In [ ]:
import sys
!{sys.executable} -m pip install sockets
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install igraph
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install pyvis

## Import Packages

In [1]:
import socket
import time

import numpy  as np
import igraph as ig
import matplotlib.pyplot as plt

plt.rc("text", usetex=True)

%run ./2-ImplementationFactor.ipynb
%run ./3-ImplementationPGM.ipynb
%run ./customizedLBP.ipynb

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


## Environment Configuration

The configuration of the environment include:
- HOST, PORT: Server setting
- ENCODING_METHOD: Encoding method for the message communicate (do not change)
- BUFFER_SIZE: Buffer size for server and client (do not change)
- END, MSG_BREAK, DILIMITER_1, DILIMITER_2, MUL_SIGN, STATEMENT_ID_PREFIX: Setting for the message (do not change)
- MAX_ITR: Maximum iteration run by the loopy belief propagation if it does not converge

In [2]:
# Server setting
HOST = "127.0.0.1"
PORT = 8080

# Encoding method
ENCODING_METHOD = "UTF-8"

# Buffer size for the message
BUFFER_SIZE = pow(2, 20)

# Setting for the message
END = "END"
MSG_BREAK = "BREAK"
DILIMITER_1 = ','
DILIMITER_2 = "&"
MUL_SIGN = '*'
STATEMENT_ID_PREFIX = "S_"

# Maximum iteration
MAX_ITR = 20

## Helper functions for server

In [3]:
def factorLoader(factor_input):
    tokens = factor_input.split(DILIMITER_2)
    for idx in range(0, len(tokens), 4):
        yield tokens[idx], tokens[idx+1], tokens[idx+2], tokens[idx+3]

In [4]:
def printGraphInput(str_):
    print("graph input:")
    str_tokens = [i.split('(') for i in str_.split(')') if i != '']
    for token in str_tokens:
        print(token[0], token[1].split(','))

In [5]:
def checkDuplicateVar(str_):
    str_tokens = [i.split('(') for i in str_.split(')') if i != '']
    for token in str_tokens:
        vars = token[1].split(',')
        if len(vars) != len(set(vars)):
            print("contain duplicate variables: ", token[0])
            print(vars)
            raise ValueError('Duplicated variables: ' + token[0])

In [6]:
def printFactorInput(factor_input):
    print("factor input:")
    tokens = factor_input.split(DILIMITER_2)
    for idx in range(0, len(tokens), 4):
        print("Node:", tokens[idx], tokens[idx+1], tokens[idx+2], tokens[idx+3])

In [7]:
def recvMsg(conn):
    graph_input = ""
    factor_input = ""
    
    # Read Graph Input
    while True:
        message = conn.recv(BUFFER_SIZE)
        if not message:
            return None, None
        message_str = message.decode(ENCODING_METHOD)
        if message_str == MSG_BREAK:
            break
        else:
            graph_input += message_str
    
    # Read Factor Input
    while True:
        message = conn.recv(BUFFER_SIZE)
        if not message:
            return None, None
        message_str = message.decode(ENCODING_METHOD)
        if message_str == MSG_BREAK:
            break
        else:
            factor_input += message_str
    
    print("graph input size: ", len(graph_input))
    print("factor input size: ", len(factor_input))
    
    return graph_input, factor_input 

In [8]:
def printNodeCount(graph):
    factor_node_count = 0
    var_node_count = 0
    
    for i in range(graph.vcount()):
        if graph.vs[i]['is_factor']:
            factor_node_count += 1
        else:
            var_node_count += 1
            
    print("var node count:", var_node_count, "factor node count:", factor_node_count, "total:", factor_node_count + var_node_count)
    print("edges count:", graph.ecount())

## Server

In [9]:
def startServer():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind((HOST, PORT))
        stillWorking = True
        while stillWorking:
            s.listen()
            conn, addr = s.accept()
            with conn:
                print(f"Connected by {addr}")

                while True:
                    print("-"*20)
                    graph_input, factor_input = recvMsg(conn)
                    
                    if not graph_input or not factor_input:
#                         print("Error: Graph input or factor input is null")
                        response = "".encode(ENCODING_METHOD)
                        conn.sendall(response)
                        break

#                     graph_input = graph_input.decode(ENCODING_METHOD)
#                     factor_input = factor_input.decode(ENCODING_METHOD)
                    
                    
#                     print("graph_input", graph_input)
#                     print("factor_input", factor_input)
#                     printGraphInput(graph_input)
#                     printFactorInput(factor_input)
        
                    checkDuplicateVar(graph_input)
                    if graph_input == END and factor_input == END:
                        output_str = END
                        response = output_str.encode(ENCODING_METHOD)
                        print("response:", response)
                        conn.sendall(response)
                        print("Terminate server...")
                        stillWorking = False
                        break

                    fg = string2factor_graph(graph_input)

                    predIDs_all = set()
                    for order, constraintID, predIDs_str, probs_str in factorLoader(factor_input):
                        try:
                            predIDs = predIDs_str.split(DILIMITER_1)

        #                     print(predIDs)
                            predIDs_all.update(predIDs)
                            predCount = len(predIDs)

                            shape = [2 for _ in range(predCount)]
                            shape = tuple(shape)

                            probs_tokens = probs_str.split(DILIMITER_1)
                            
                            probs = []
                            for probs_token in probs_tokens:
                                probs_str, count = probs_token.split(MUL_SIGN)
                                count = int(count)
                                prob = float(probs_str)
                                for _ in range(count):
                                    probs.append(prob)
                                    
                            probs = np.array(probs)
                            probs = probs.reshape(shape)

                            fg.change_factor_distribution(constraintID, factor(predIDs,  probs))
                        except Exception as e:
                            print(e)
                            print("constraintID:", constraintID)
                            print("predIDs_str:", predIDs_str)
                            print("probs_str:", probs_str)
                            return

                    printNodeCount(fg.get_graph())
                            
                    lbp = myLBP(fg)
                    start = time.time()
                    margProb = lbp.belief(predIDs_all, MAX_ITR)
                    end = time.time()
                    
                    print("time needed: ", end - start)
                    plot_factor_graph(fg)
                    output_str = ""
                    for predID, prob in margProb.items():
#                         print(predID, prob)
                        output_str += predID + DILIMITER_1 + str(prob)
                        output_str += DILIMITER_2

                    output_str = output_str[:-1]
                    response = output_str.encode(ENCODING_METHOD)
#                     print("response:", response)
                    conn.sendall(response)

In [ ]:
startServer()

Connected by ('127.0.0.1', 51102)
--------------------
graph input size:  1256
factor input size:  1467
var node count: 4 factor node count: 10 total: 14
edges count: 20
time needed:  0.38801050186157227
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
--------------------
Connected by ('127.0.0.1', 51103)
--------------------
graph input size:  181
factor input size:  243
var node count: 2 factor node count: 3 total: 5
edges count: 5
time needed:  0.008428096771240234
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
--------------------
Connected by ('127.0.0.1', 51104)
--------------------
graph input size:  736
factor input size:  909
var node count: 4 factor node count: 6 total: 10
edges count: 15
time needed:  0.1286332607269287
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
--------------------
Connected by ('127.0.0.1', 51105)
--------------------
graph input size:  801
factor